# RUSLE

Adapted from code by [Lucas Rivero Iribarne](https://www.lucasriveroiribarne.com).

In [1]:
# =============================================================================
# Environment Setup (using uv)
# =============================================================================
# First time setup: Run in terminal from project root:
#   uv sync
#
# This installs all dependencies from pyproject.toml into a virtual environment.
# Then select the .venv kernel in VS Code/Jupyter.
#
# If running in notebook and packages are missing, uncomment below:
# !uv pip install earthengine-api geemap geopandas matplotlib numpy pandas seaborn rasterio branca

import sys
print(f"Python: {sys.executable}")
print(f"Version: {sys.version}")

Python: /Users/nico/Desktop/Projects/erosion_models/RUSLE/.venv/bin/python
Version: 3.12.11 (main, Sep  2 2025, 14:12:30) [Clang 20.1.4 ]


## Spatial dataset

### Setup

In [2]:
"""Load libraries and initialize Google Earth Engine API."""

import math
import os

import ee
import geemap
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Import local modules
import rusle_utils as rusle
from gee_auth import initialize_gee, print_auth_status, setup_project

In [3]:
# =============================================================================
# Google Earth Engine Authentication (Automatic & Persistent)
# =============================================================================
# First time or after credential issues: force re-authentication
# This will open a browser window for you to sign in

# Your GEE Cloud Project ID
GEE_PROJECT = 'ee-nriveras'

# Force fresh authentication (set to True if having issues)
FORCE_AUTH = True

import ee
from gee_auth import clear_credentials

# 1. Clear old credentials if forcing auth
if FORCE_AUTH:
    print("Clearing old credentials...")
    clear_credentials()

# 2. Authenticate (Force new login)
print("Authenticating... (Link will appear if needed)")
ee.Authenticate(force=True)

# 3. Initialize Project
print(f"Initializing project: {GEE_PROJECT}...")
try:
    ee.Initialize(project=GEE_PROJECT)
    print(f"✓ Google Earth Engine initialized successfully!")
    
    # Test connection
    info = ee.Number(1).getInfo()
    print("✓ Connection verified.")
    
except ee.EEException as e:
    print(f"\n✗ Initialization Failed: {e}")
    if "not registered" in str(e):
        print(f"\nACTION REQUIRED: Project '{GEE_PROJECT}' is not registered for Earth Engine.")
        print(f"1. Register it here: https://console.cloud.google.com/earth-engine/configuration?project={GEE_PROJECT}")
        print("2. OR update GEE_PROJECT variable above with a valid Cloud Project ID.")
    elif "permission" in str(e).lower():
        print(f"\nACTION REQUIRED: You may not have permissions for project '{GEE_PROJECT}'.")
        print("Check your Google Cloud Console IAM settings.")
except Exception as e:
    print(f"\n✗ An unexpected error occurred: {e}")

Clearing old credentials...
✓ Credentials removed. You will need to re-authenticate.
Authenticating... (Link will appear if needed)



Successfully saved authorization token.
Initializing project: ee-nriveras...
✓ Google Earth Engine initialized successfully!
✓ Connection verified.


### Load Area of Interest

In [132]:
# =============================================================================
# Configuration Parameters
# =============================================================================

# Analysis period
DATE_FROM = '2022-11-14'
DATE_TO = '2023-11-14'

# Study area
ADMIN_REGION = 'Metropolitana'

# Output settings
OUTPUT_DIR = '../03_output'
EXPORT_SCALE = 10000  # meters

In [133]:
# Load area of interest from FAO GAUL administrative boundaries
aoi = rusle.load_area_of_interest(ADMIN_REGION, admin_level=1)

# Alternative: Load from local shapefile
# chile = gpd.read_file('../02_input/Regiones/Regional.shp')
# region_metropolitana = chile[chile['codregion'] == 13]
# aoi = geemap.geopandas_to_ee(region_metropolitana)

## Data Collection

### Precipitation 

In [134]:
# Load CHIRPS precipitation data
# Source: https://developers.google.com/earth-engine/datasets/catalog/UCSB-CHG_CHIRPS_DAILY
precipitation = rusle.load_precipitation_data(aoi)

### Soil

Carbon, sand, silt, and clay are provided by the OpenLandMap project at a global scale based on machine learning predictions from soil profile and sample compilations.

In [135]:
# Load soil data from OpenLandMap
# Surface layer (b0) only - erosion is a surface process
organic_carbon, clay, sand, silt = rusle.load_soil_data(aoi)

### Digital Elevation Model

In [136]:
# Load Digital Elevation Model
dem_srtm = rusle.load_dem('SRTM')
dem_merit = rusle.load_dem('MERIT')  # Note: MERIT has limited global coverage

### Landsat 8

In [137]:
# Load Landsat 8 Surface Reflectance with scaling factors applied
landsat8 = rusle.load_landsat8()

### Landcover

In [138]:
# Load MODIS land cover (most recent available)
modis_landcover = rusle.load_modis_landcover(aoi)

## RUSLE Factors
### Soil Erodibility (K)

The following implementation was based on [this tutorial](https://youtu.be/S6RR-pW6hSE?si=d0ZuYPAnsfWUrffH) which uses as reference the method proposed by [Williams (1995)](https://swat.tamu.edu/media/99192/swat2009-theory.pdf). Williams proposes that:

$$
K_{USLE} = f_{csand}\cdot f_{cl-si}\cdot f_{orgc}\cdot f_{hisand}
$$

Where $f_{csand}$ is a factor that gives low soil erodibility factors for soils with high coarse sand content and high values for soils with little sand, $f_{cl-si}$ is a factor that gives low soil erodibility factors for soils with high clay-silt ratios, $f_{orgc}$ is a factor that reduces soil erodibility for soils with high organic carbon content, and $f_{hisand}$ is a factor that reduces soil erodibility for soils with extremely high sand content. The factors are calculated as:

$$
\begin{equation}
\begin{aligned}
f_{csand} &= \left(0.2+0.3\cdot exp\left[-0.256 \cdot m_s \cdot \left(1- \dfrac{m_{silt}}{100}\right)\right]\right)
\end{aligned}
\end{equation}
\\
\begin{equation}
\begin{aligned}
f_{cl-si} &= \left( \dfrac{m_{silt}}{m_{c}+m_{silt}}\right)^{0.3}
\end{aligned}
\end{equation}
\\
\begin{equation}
\begin{aligned}
f_{orgc} &= \left(1- \dfrac{0.25 \cdot orgC}{orgC + exp[3.72-2.95 \cdot orgC]}\right)
\end{aligned}
\end{equation}
\\
\begin{equation}
\begin{aligned}
f_{hisand} &= \left(1- \dfrac{0.7 \cdot \left(1- \dfrac{m_s}{100}\right)}{\left(1- \dfrac{m_s}{100}\right)+exp\left[ -5.51+22.9 \cdot \left(1- \dfrac{m_s}{100}\right)\right]}\right)
\end{aligned}
\end{equation}
$$

where $m_s$ is the percentage content of sand (particles 0.05-2.00 mm in diameter), $m_{silt}$ is the percentage content of silt (particles 0.002-0.05 mm in diameter), $m_c$ is the percentage content of clay (particles < 0.002 mm in diameter), and $orgC$ is the percentage content of organic carbon in the layer (%).


In [139]:
# Calculate K factor (Soil Erodibility) using Williams (1995) method
k_factor = rusle.calculate_k_factor(
    sand=sand,
    silt=silt,
    clay=clay,
    c_org=organic_carbon
)

### Rainfall Erosivity (R)

Rainfall erosivity is the estimation of soil loss caused by precipitation (Uddin et al. 2018).

$R = 0.0483 * P^{1.610}$

Where P corresponds to annual precipitation

In [140]:
# Calculate R factor (Rainfall Erosivity)
# Note: Ideally should use multi-year average for better representation
r_factor = rusle.calculate_r_factor(
    precipitation=precipitation,
    date_from=DATE_FROM,
    date_to=DATE_TO,
    aoi=aoi
)

The values of the R factors seems to be to hight in comparison with the other factors. Is this correct?

### Slope Length (L)

The slope length represents the distance from a given pixel to the potential erosion point.

$L = (\frac{λ}{22.13})^m$

Where λ corresponds to the slope length and *m* takes a value between 0.2 and 0.5 depending on the slope.

In [141]:
# Calculate slope metrics from DEM
slope_deg, slope_perc = rusle.calculate_slope_metrics(dem_srtm, aoi)

In [142]:
# Calculate L factor (Slope Length)
l_factor = rusle.calculate_l_factor(slope_perc, pixel_size=30.0)

### Slope Steepness (S)

The slope steepness factor represents the rate at which water can flow on a given surface, interacting with the soil angle and affecting the magnitude of soil erosion.

$S = \frac{(0.43 + 0.3 * S + 0.043 * S^2)}{6.613}$

Where *S* corresponds to slope in percentage.

In [143]:
# Calculate S factor (Slope Steepness)
# Note: Values may appear high for steep terrain - this is expected
s_factor = rusle.calculate_s_factor(slope_perc)

### Vegetation Cover (C)

De Jong (1994) developed the following relationship between field-calibrated C factor values with the Landsat-based Normalized Difference Vegetation Index (NDVI) to produce a continuous C factor surface.

$C = 0.431 - 0.805 * NDVI$

$NDVI = \frac{NIR - Red}{NIR + Red}$

Where NIR corresponds to reflectivity in the near infrared and Red corresponds to reflectivity in the red band.

In [144]:
# Calculate C factor (Vegetation Cover) using De Jong (1994) method
c_factor = rusle.calculate_c_factor(
    landsat=landsat8,
    date_from=DATE_FROM,
    date_to=DATE_TO,
    aoi=aoi
)

### Erosion Control Practices (P)

The erosion control practice factor reflects the impact of support practices on the average annual erosion rate, and also describes the relationship between soil losses in a specific field where erosion control practice is determined.

In this exercise, the MODIS land cover product was considered (the most updated land use version of this dataset is 2020) and a value was assigned for each land cover based on Chuenchum et al., 2019.


In [145]:
# Calculate P factor (Erosion Control Practices)
# Using MODIS land cover with values from Chuenchum et al., 2019
modis_lc_full = (
    ee.ImageCollection("MODIS/006/MCD12Q1")
    .select('LC_Type1')
    .sort('system:time_start', False)
    .first()
)

p_factor = rusle.calculate_p_factor(
    modis_lc=modis_lc_full,
    aoi=aoi
)

## RUSLE Model

In [146]:
# =============================================================================
# Calculate RUSLE Soil Loss: A = R * K * L * S * C * P
# =============================================================================

soil_loss = rusle.calculate_rusle(
    r_factor=r_factor,
    k_factor=k_factor,
    l_factor=l_factor,
    s_factor=s_factor,
    c_factor=c_factor,
    p_factor=p_factor,
    pixel_area_ha=0.09  # 30m x 30m pixel = 0.09 ha
)

In [147]:
# =============================================================================
# Interactive Map with All Layers (toggle layers in layer control panel)
# =============================================================================

# Create single map with all layers
final_map = geemap.Map()

# --- Base Data Layers (initially hidden) ---
# Area of Interest
final_map.addLayer(aoi, {'color': 'black'}, 'Area of Interest', shown=True)

# DEM
dem_viz = {'min': 0, 'max': 4000, 'palette': ['green', 'yellow', 'brown', 'white']}
final_map.addLayer(dem_srtm.clip(aoi), dem_viz, 'DEM (SRTM)', shown=False)

# Landsat 8 RGB
rgb_viz = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0.0, 'max': 0.3}
final_map.addLayer(landsat8.median().clip(aoi), rgb_viz, 'Landsat 8 RGB', shown=False)

# MODIS Land Cover
igbp_palette = [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
]
lc_viz = {'min': 1, 'max': 17, 'palette': igbp_palette}
final_map.addLayer(modis_landcover, lc_viz, 'MODIS Land Cover', shown=False)

# Soil Properties
soil_viz = {'min': 0, 'max': 100, 'palette': ['white', 'brown', 'black']}
final_map.addLayer(organic_carbon.clip(aoi), soil_viz, 'Organic Carbon (%)', shown=False)
final_map.addLayer(clay.clip(aoi), soil_viz, 'Clay (%)', shown=False)
final_map.addLayer(silt.clip(aoi), soil_viz, 'Silt (%)', shown=False)
final_map.addLayer(sand.clip(aoi), soil_viz, 'Sand (%)', shown=False)

# --- RUSLE Factor Layers (initially hidden) ---
factor_palette = ['blue', 'green', 'yellow', 'orange', 'red']

# K Factor (Soil Erodibility)
k_viz = {'min': 0.3, 'max': 0.5, 'palette': factor_palette}
final_map.addLayer(k_factor.clip(aoi), k_viz, 'K Factor (Soil Erodibility)', shown=False)

# R Factor (Rainfall Erosivity)
r_viz = {'min': 0, 'max': 5000, 'palette': factor_palette}
final_map.addLayer(r_factor.clip(aoi), r_viz, 'R Factor (Rainfall Erosivity)', shown=False)

# L Factor (Slope Length)
l_viz = {'min': 1, 'max': 1.2, 'palette': rusle.SLOPE_PALETTE}
final_map.addLayer(l_factor.clip(aoi), l_viz, 'L Factor (Slope Length)', shown=False)

# S Factor (Slope Steepness)
s_viz = {'min': 0, 'max': 45, 'palette': rusle.SLOPE_PALETTE}
final_map.addLayer(s_factor.clip(aoi), s_viz, 'S Factor (Slope Steepness)', shown=False)

# C Factor (Vegetation Cover)
c_viz = {'min': 0, 'max': 0.5, 'palette': rusle.SLOPE_PALETTE}
final_map.addLayer(c_factor.clip(aoi), c_viz, 'C Factor (Vegetation Cover)', shown=False)

# P Factor (Erosion Control)
p_viz = {'min': 0, 'max': 1, 'palette': factor_palette}
final_map.addLayer(p_factor.clip(aoi), p_viz, 'P Factor (Erosion Control)', shown=False)

# --- Final Result Layer (shown by default) ---
soil_loss_viz = {'min': 0, 'max': 50, 'bands': ['soil_loss'], 'palette': [
    '00ff00', '7fff00', 'ffff00', 'ffa500', 'ff4500', 'ff0000', '8b0000'
]}
final_map.addLayer(soil_loss.clip(aoi), soil_loss_viz, 'RUSLE Soil Loss (ton/ha/yr)', shown=True)

# Add colorbar (pass palette directly without 'bands' key)
colorbar_params = {'min': 0, 'max': 50, 'palette': [
    '00ff00', '7fff00', 'ffff00', 'ffa500', 'ff4500', 'ff0000', '8b0000'
]}
final_map.add_colorbar_branca(
    colors=colorbar_params['palette'],
    vmin=colorbar_params['min'],
    vmax=colorbar_params['max'],
    caption='Soil Loss (ton/ha/year)'
)

# Center and display
final_map.centerObject(aoi, 9)
print("Use the layer control (top-right) to toggle layers on/off")
final_map

Use the layer control (top-right) to toggle layers on/off


Map(center=[-33.60281244992525, -70.62386347471441], controls=(WidgetControl(options=['position', 'transparent…

In [148]:
# Export soil loss raster to local file (only if not already exists)
# Include date range and AOI in filename for identification
output_filename = f"RUSLE_soil_loss_{ADMIN_REGION}_{DATE_FROM}_to_{DATE_TO}.tif"
output_path = f'{OUTPUT_DIR}/{output_filename}'

if os.path.exists(output_path):
    print(f"✓ File already exists, skipping export: {output_path}")
else:
    print(f"Exporting soil loss raster...")
    rusle.export_image(
        image=soil_loss,
        output_path=output_path,
        aoi=aoi,
        scale=EXPORT_SCALE
    )
    print(f"✓ Exported soil loss raster to: {output_path}")

# Generate preview image for README
preview_path = f'{OUTPUT_DIR}/{output_filename.replace(".tif", "_preview.jpg")}'

if os.path.exists(output_path):
    if not os.path.exists(preview_path):
        print(f"Generating preview image...")
        
        import rasterio
        
        # Read GeoTIFF with rasterio
        with rasterio.open(output_path) as src:
            data = src.read(1)
        
        # Create figure
        fig, ax = plt.subplots(figsize=(10, 10))
        
        # Plot with same colormap as interactive map
        from matplotlib.colors import LinearSegmentedColormap
        colors = ['#00ff00', '#7fff00', '#ffff00', '#ffa500', '#ff4500', '#ff0000', '#8b0000']
        cmap = LinearSegmentedColormap.from_list('soil_loss', colors)
        
        # Mask nodata values
        masked_data = np.ma.masked_where((data < 0) | (data > 500), data)
        
        im = ax.imshow(masked_data, cmap=cmap, vmin=0, vmax=50)
        ax.set_title(f'RUSLE Soil Loss - {ADMIN_REGION}\n{DATE_FROM} to {DATE_TO}', fontsize=14)
        ax.axis('off')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=ax, shrink=0.7, label='Soil Loss (ton/ha/year)')
        
        # Save as JPEG
        plt.savefig(preview_path, dpi=150, bbox_inches='tight', 
                   format='jpeg', facecolor='white')
        plt.close()
        
        print(f"✓ Preview saved to: {preview_path}")
    else:
        print(f"✓ Preview already exists: {preview_path}")

✓ File already exists, skipping export: ../03_output/RUSLE_soil_loss_Metropolitana_2022-11-14_to_2023-11-14.tif
✓ Preview already exists: ../03_output/RUSLE_soil_loss_Metropolitana_2022-11-14_to_2023-11-14_preview.jpg


## References

1. Uddin, K., Abdul Matin, M., & Maharjan, S. (2018). Assessment of land cover change and its impact on changes in soil erosion risk in Nepal. Sustainability, 10(12), 4715.

2. Renard, K. G. (1997). Predicting soil erosion by water: a guide to conservation planning with the Revised Universal Soil Loss Equation (RUSLE). United States Government Printing.

3. Funk, C., Peterson, P., Landsfeld, M., Pedreros, D., Verdin, J., Shukla, S., ... & Michaelsen, J. (2015). The climate hazards infrared precipitation with stations—a new environmental record for monitoring extremes. Scientific data, 2(1), 1-21.

4. Boisier, J. P., Alvarez-Garretón, C., Cepeda, J., Osses, A., Vásquez, N., & Rondanelli, R. (2018, April). CR2MET: A high-resolution precipitation and temperature dataset for hydroclimatic research in Chile. In EGU General Assembly Conference Abstracts (p. 19739).

5. Zambrano, F., Wardlow, B., Tadesse, T., Lillo-Saavedra, M., & Lagos, O. (2017). Evaluating satellite-derived long-term historical precipitation datasets for drought monitoring in Chile. Atmospheric Research, 186, 26-42.

6. De Jong, S. M. (1994). Derivation of vegetative variables from a Landsat TM image for modelling soil erosion. Earth Surface Processes and Landforms, 19(2), 165-178.

7. Chuenchum, P., Xu, M., & Tang, W. (2019). Estimation of soil erosion and sediment yield in the Lancang–Mekong river using the modified revised universal soil loss equation and GIS techniques. Water, 12(1), 135.